# 1. Sample demo 

In [0]:
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

In [0]:
# create computing graph
gl = tf.get_default_graph() #default graph
w = tf.constant(2.)
y = w+2

In [0]:
# load session
with tf.Session(graph=gl) as sess:
  print(sess.run([y]))

[4.0]


In [0]:
# clear graph
tf.reset_default_graph()

# 2. Basics of Tensorflow  
## concepts:

- graph
- session
- op
- tensor
- variable
- placeholder
- path
- tf.assign

### graph 
In Tensorflow, edge represents the direction of the flows,node represents tensor and op.

In [0]:
# when open tensorflow, tf 会分配一个 default graph, all the op would run in this default graph.
g0 = tf.get_default_graph()
x0 = tf.Variable(1) # graph 的构建
x0.graph is g0 # check graph 的构建 属不属于这个图

True

In [0]:
# create two graphs
g1 = tf.Graph()
g2 = tf.Graph()

# operations on g1 
with g1.as_default():
  x1 = tf.Variable(1)

with g2.as_default():
  x2 = tf.Variable(1)

print(x1.graph is g2)
print(x2.graph is g2)


False
True


In [0]:
# get all the nodes in one graph
g1.get_operations()

[<tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>]

In [0]:
# 当构建 graph 的时候，可能会由于重复构建导致 graph 出错，so have to clear the default graph when creating new graphs
g_now = tf.get_default_graph()
g_now.get_operations()


[<tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>]

In [0]:
# clear the default graphs 
tf.reset_default_graph()
g_now = tf.get_default_graph()
g_now.get_operations()

[]

Tensorflow 是一种静态图的学习框架，2.0 开始有动态图，Pytorch 是一种动态图的框架。动态图的好处是更符合人的编程习惯。未来的趋势是静态图、动态图相互切换。

### session
graphs don't take much resources, but session take a lot resources.


In [0]:
w = tf.constant(3)
x = w+2
y = x +5
z = x*3

with tf.Session(graph = tf.get_default_graph()) as sess:
  print(sess.run([x]))
  print(sess.run([z]))
  print(x.eval())
  

[5]
[15]
5


Tips:
1. eval() = sess.run()
2. tendorflow 会自动检测依赖关系
3. 除了 variable,其余计算结果每次计算后会释放。variable 会在 session 结束后释放。
4. 重复计算的问题。比如run 完 x 就会被释放，但run z 时需要重复计算 x; 解决方法是把 xz session 放在一起。

In [0]:
with tf.Session(graph = tf.get_default_graph()) as sess:
  print(sess.run([x,z]))

In [0]:
# 每次使用上下文管理器太麻烦，于是我们使用有互动的session, real time feedback.
# need to close by hand after using
sess = tf.InteractiveSession()
print(x.eval())
print(sess.run([x]))

5
[5]


In [0]:
sess.close()
# variables would be released at this point.

### Tensor
There are three main sources of tensor:
- constant (生存周期在 session 内)
- variable 
- placeholder

`tf.constant()` make new constant

In [0]:
import numpy as np
a = tf.constant(np.arange(12).reshape(3,4),dtype=tf.float32)
with tf.Session(graph = tf.get_default_graph())as sess:
  print(sess.run([a]))
tf.reset_default_graph()

[array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]], dtype=float32)]


In [0]:
a = tf.constant(3.14, shape=(3,4),dtype=tf.float32)
with tf.Session(graph = tf.get_default_graph())as sess:
  print(sess.run([a]))
tf.reset_default_graph()

[array([[3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14]], dtype=float32)]


### Variable
Variable is a kind of tensor. It could be saved and updated in the session. 

⭐ Variables needed to be initialized, demo as following:

```
init = tf.globle_Variables_initializer() // 就不需要一个一个地初始化了
sess.run(init)
```
Generally, `init` is created in the graph.
Remark: Tndorflow usually uses `tf.Variable` and `tf.get_variable`, but these have huge differences. 尽量使用 `tf.get_variable`.

### `tf.Variable`
1. 每次调用得到的都是不同的变量，即使使用了相同的变量名，在底层实现的时候还是会为变量创建不同的别名。

In [0]:
var1 = tf.Variable(tf.random_uniform([1],-1.0,1.0),name='var',dtype=tf.float32)
var2 = tf.Variable(initial_value=[2],name='var',dtype=tf.float32)
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  print(var1.name,sess.run(var1))
  print(var2.name,sess.run(var2))

var_2:0 [0.9093454]
var_3:0 [2.]


2. 会受`tf.name_scope` 环境的影响：会在前面加上 `name_scope` 的空间前缀。

In [0]:
tf.reset_default_graph()
with tf.name_scope('var_b_scope'):
  var1 = tf.Variable(initial_value=[2],name='var',dtype=tf.float32)
  var2 = tf.Variable(initial_value=[2],name='var',dtype=tf.float32)
with tf.name_scope('var_a_scope'):
  var3 = tf.Variable(initial_value=[2],name='var',dtype=tf.float32)
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  print(var1.name,sess.run(var1))
  print(var2.name,sess.run(var2))
  print(var3.name,sess.run(var3))

var_b_scope/var:0 [2.]
var_b_scope/var_1:0 [2.]
var_a_scope/var:0 [2.]


3. `Variable()` 在创建时可以直接指定初始化的方式，还可以把其他变量的初始值当作初始值。

In [0]:
var2 = tf.Variable(var1.initialized_value())

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


### `tf.get_variable()`
1. 只会创建一个同名的变量，如果想共享变量，需指定 `reuse = True`, 否则多次创建会导致报错。使用 `reuse = True`(not needed for the first time creating new vaviable,但须在后面共享的时候claim.)可以动态地修改某个 `scope` 的共享属性。

    为什么需要共享变量？因为卷积里有一个很重要的特点是共享参数。


In [0]:
tf.reset_default_graph()
def func(x):
  weight= tf.get_variable(name='weight',initializer = tf.random_normal([1]))
  bias = tf.get_variable(name = 'bias', initializer =tf.zeros([1]))
  return tf.add(tf.multiply(weight,x),bias)

result1 = func(1)
#result2 = func(2)
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  print(sess.run([result1]))
  #print(sess.run([result2]))
  
## problem: weight and bias would run repeatly, which would cause error in tf.get_variable()

[array([-0.5048192], dtype=float32)]


In [0]:
tf.reset_default_graph()
def func(x,reuse):
  with tf.variable_scope('neuron',reuse = reuse): //neoron 相当于前面 var_a 的前缀: var
    weight= tf.get_variable(name='weight',initializer = tf.random_normal([1]))
    bias = tf.get_variable(name = 'bias', initializer =tf.zeros([1]))
  return tf.add(tf.multiply(weight,x),bias)

result1 = func(1,reuse = False)
result2 = func(2,reuse = True) //should be double of result1
init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  print(sess.run([result1]))
  print(sess.run([result2]))

[array([0.14972697], dtype=float32)]
[array([0.29945394], dtype=float32)]


2. 不受 `with tf.name_scope` 的影响（是`name_scope`, not `variable_scope`; both `tf.variable` and `tf.get_variable` 都会受 `variable_scope`的影响）。

In [0]:
tf.reset_default_graph()
with tf.name_scope('var_a_scope'):
  var1 = tf.get_variable(name='var',shape= [1],dtype = tf.float32)
  var2 = tf.get_variable(name='var1',shape= [1],dtype = tf.float32)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run([var1]))
  print(sess.run([var2]))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[array([-0.01262069], dtype=float32)]
[array([1.5309323], dtype=float32)]


3. initialization
4. `with tf.name_scope('scope name')` 会进行“累加”：每调用一次就会给里面的所有变量增加一个前缀，叠加顺序是 外层先调用的在前，后调用的在后。  

### placeholder
we need something to receive the data from outside: placeholder.
BatchSize 是占了个位置，占好位置后，输入的数据在不断变化。
Remark:
1. placeholder is a type of tensor too.
2. input datatype usually is `ndarray` from `numpy`

In [0]:
import numpy as np
ph = tf.placeholder(dtype=tf.float32,shape=(None,3)) # None 那个位置其实就是一个 batchsize, which is changable.
add_op = tf.add(ph,1)
with tf.Session() as sess:
  print(sess.run(add_op,feed_dict = {ph:np.random.rand(5,3)}))


[[1.3998804 1.6061504 1.5851212]
 [1.7210946 1.5154355 1.2905102]
 [1.7051034 1.0635133 1.5128222]
 [1.6410382 1.0614496 1.8534691]
 [1.6128211 1.5241439 1.1005969]]


### 计算路径

计算路径是指如果计算的 node 具有依赖关系，那么我们就会计算这些 node，沿着father node 寻找。Tensorflow 仅通过必需的节点自动进行计算 是这个框架的巨大优势。if there's a large graph with a lot unnecessary nodes,tensorflow would save a lot computation and time. 它允许我们构建大型的多用途graph, 这些graph 使用单个共享的核心node 集合，并根据所采取的不同的计算路径去做不同的事情。它只会关心它觉得重要/必要的路径。

In [0]:
# tensorflow会自动寻找依赖关系
# 如果去掉feed_dict会报错
tf.reset_default_graph()

ph = tf.placeholder(tf.int32)

three_node = tf.constant(3)

sum_node = ph + three_node

with tf.Session() as sess:
    #print(sess.run(three_node))
    print(sess.run(sum_node,feed_dict={ph:15}))

### `tf.assign`


`tf.assign(target, value)`表示把`value`值赋值给`target`。`target`必须是一个可变的`tensor`(variable)可以没被初始化。`value`必须要有和`target`相同的数据类型和形状。

思考一下如下的操作需要用到`tf.assign`吗？如果要用，对谁用？

$\theta = \theta - \beta \nabla L(\theta)$  ## 梯度向量法
--> tf.assign($\theta$ , $\theta - \beta \nabla L(\theta)$)

---

## Logistic Regression

线性回归可以看作是最简单的神经网络。我们使用4种方法来实现一个线性回归。

- 解析法。
- 人工求梯度。
- 使用低阶API求梯度。
- 使用高阶API求梯度。

最常用3、4 方法。
  |  
  |  
  v 
  
 easier

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess: 
    theta_value = theta.eval() ## have to assign the theta to a variable, since it would be released after the session.
    print(theta_value)

[[-3.67372932e+01]
 [ 4.37366009e-01]
 [ 9.47520509e-03]
 [-1.08159676e-01]
 [ 6.48537397e-01]
 [-3.84734449e-06]
 [-3.79239232e-03]
 [-4.19136107e-01]
 [-4.32144403e-01]]


In [2]:
import time
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

data = housing.data
scaled_housing_data_plus_bias = (data-np.mean(data,axis=0))/np.std(data,axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data_plus_bias]

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")



theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse") # 相当于loss
# 对 loss 进行梯度下降

gradients = 2./m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer() 

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0: #批量梯度下降
            print("Epoch", epoch, "MSE =", mse.eval())
            time.sleep(1)
        sess.run(training_op)
    best_theta = theta.eval() # save theta
    print(best_theta)

Epoch 0 MSE = 4.855082
Epoch 100 MSE = 0.6381838
Epoch 200 MSE = 0.5704652
Epoch 300 MSE = 0.55751604
Epoch 400 MSE = 0.5488386
Epoch 500 MSE = 0.5425051
Epoch 600 MSE = 0.5378545
Epoch 700 MSE = 0.5344304
Epoch 800 MSE = 0.53190255
Epoch 900 MSE = 0.530031
[[ 2.0685523 ]
 [ 0.8420356 ]
 [ 0.14195044]
 [-0.24994995]
 [ 0.27564868]
 [ 0.00377547]
 [-0.04155352]
 [-0.7191171 ]
 [-0.6893291 ]]


In [3]:
tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01

data = housing.data
scaled_housing_data_plus_bias = (data-np.mean(data,axis=0))/np.std(data,axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data_plus_bias]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse") #loss

gradients = tf.gradients(mse,theta) # loss 关于 theta 的梯度
#前一个 gradients 是人求的，这一个是tf求的

training_op = tf.assign(theta, theta - learning_rate * gradients[0])
init = tf.global_variables_initializer() 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch", epoch, "MSE =", mse.eval())
            time.sleep(1)
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE = 7.5094023
Epoch 100 MSE = 0.72326267
Epoch 200 MSE = 0.5605887
Epoch 300 MSE = 0.5483131
Epoch 400 MSE = 0.5428546
Epoch 500 MSE = 0.53887355
Epoch 600 MSE = 0.5358149
Epoch 700 MSE = 0.5334431
Epoch 800 MSE = 0.5315937
Epoch 900 MSE = 0.53014416
[[ 2.0685523 ]
 [ 0.7420637 ]
 [ 0.11559272]
 [-0.07530657]
 [ 0.13749456]
 [-0.0045165 ]
 [-0.03719198]
 [-1.0051216 ]
 [-0.96438694]]


In [4]:
## high-level API usage

tf.reset_default_graph()
n_epochs = 1000
learning_rate = 0.01

data = housing.data
scaled_housing_data_plus_bias = (data-np.mean(data,axis=0))/np.std(data,axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data_plus_bias]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)# use momentum instead of gradient
training_op = optimizer.minimize(mse) #optimize mse

init = tf.global_variables_initializer() 
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch", epoch, "MSE =", mse.eval())
            time.sleep(1)
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)

Epoch 0 MSE = 6.6729083
Epoch 100 MSE = 0.53904
Epoch 200 MSE = 0.52482295
Epoch 300 MSE = 0.5243623
Epoch 400 MSE = 0.524326
Epoch 500 MSE = 0.5243217
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.524321
Epoch 900 MSE = 0.524321
[[ 2.0685577 ]
 [ 0.82962555]
 [ 0.11875281]
 [-0.26553842]
 [ 0.30570585]
 [-0.00450268]
 [-0.0393265 ]
 [-0.8998717 ]
 [-0.8705278 ]]


## Save model 
A checkpoint for models.

在模型的参数学习过程中，我们需要根据情况保存模型。根据前面的讲解知道神经网络的参数存储在`variable`中，`variable`的参数在`session`关闭后就会释放。所以我们需要在`session`打开的时候保存模型的参数。

保存模型参数类似于`checkpoint`(切片快照)。在迭代的过程中，选择某一次快照一下然后保存到硬盘中。

> 注意：模型保存在硬盘中有四个文件

In [0]:
tf.reset_default_graph()
theta = tf.Variable(tf.random_uniform([3, 1], -1.0, 1.0), name="theta")
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 1000
with tf.Session() as sess: 
    sess.run(init)
    for epoch in range(n_epochs):
        # checkpoint every 100 epochs
        if epoch % 100 == 0: 
            saver.save(sess, save_path="./model/my_model.ckpt") #save models to local
    print(theta.eval())


文件中主要保存了两类东西：

1. 计算图(保存在meta文件中)
2. variable的参数(保存在data文件中)

---
Two ways to load model we saved:

1. 复制之前的代码，生成一摸一样的计算图，然后加载参数。
2. 加载meta文件，将计算图加载回来，然后加载参数。

In [0]:
# 1st way
tf.reset_default_graph()
theta = tf.Variable(tf.random_uniform([3, 1], -1.0, 1.0), name="theta")
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess: 
    saver.restore(sess,save_path="./model/my_model.ckpt")
    print(theta.eval())

In [0]:
# 2nd way: load the graph directly. 
# more convinient, but we don't know the paramters, like theta

tf.reset_default_graph()
saver = tf.train.import_meta_graph('./model/my_model.ckpt.meta')
with tf.Session() as sess:
    saver.restore(sess,'./model/my_model.ckpt')

###  tf.get_collection vs tf.add_to_collection

为了方便我们取出不同的operation，我们需要使用tf.add_to_collection和tf.get_collection.

In [0]:
tf.reset_default_graph()
theta = tf.Variable(tf.random_uniform([3, 1], -1.0, 1.0), name="theta")
tf.add_to_collection('my_op',theta)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 1000
with tf.Session() as sess: 
    sess.run(init)
    for epoch in range(n_epochs):
        # checkpoint every 100 epochs
        if epoch % 100 == 0: 
            saver.save(sess, save_path="./model/my_model.ckpt")
    print(theta.eval())

In [0]:
tf.reset_default_graph()
saver = tf.train.import_meta_graph('./model/my_model.ckpt.meta')
my_op = tf.get_collection('my_op')
with tf.Session() as sess:
    saver.restore(sess,'./model/my_model.ckpt')
    print(sess.run(my_op[0]))

---

## TensorBoard

TensorBoard is a visualization tool for tensorflow

大致流程如下：

1. 在你创建的图里面，选择你要汇总(summary)的节点。
2. 因为你要对每一个汇总操作,进行sess.run，为了方便所以我们需要将所有操作进行汇总。(tf.summary.merge_all())。// 一次性操作
3. 在sess中运行上面汇总的操作。
4. use `tf.summary.FileWriter`,write the results into the file
5. use tensorboard --logdir='path' to run the file


### summary

---
```python
#统计标量，比如loss,accuracy，得到时序图
tf.summary.scalar(name,tensor)
```
---

```python
#统计张量，直方图统计，看weights,bias的分布
tf.summary.histogram(name,tensor)
```
---

```python
# 将summary的操作进行汇总
# inputs是个list
# 一个表示部分汇总一个表示全部汇总
merge_some=tf.summary.merge(inputs,collections=None,name=None)
merge_summary=tf.summary.merge_all(key=tf.GraphKeys.SUMMARIES)
```
---

Remark: 
1. All of the operation above happen in the defined graph.


### 文件写入操作

---

```python
#写入到硬盘的文件
#这个操作把图写入文件中
file_writer=tf.summary.FileWriter(logdir,graph,flush_secs)
```
---

```python
merge=sess.run(merge_some)
file_writer.add_summary(merge,step)
```

注意，该操作是在sess会话中运行

```python
[...]
for batch_index in range(n_batches):
    X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size) 
    if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
    sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
[...]
```


 ---
### run tensorboard

```bash
#path为目录地址
tensorboard --logdir='path'
```

```bash
#关于端口被占用的解决方法
#default port is 6006
lsof -i:6006
kill -9 4969
```


```
#在浏览器中输入
 http://0.0.0.0:6006/ (or http://localhost:6006/)
```

6006倒过来就是goog的意思


In [0]:
# tensorboard举例
import tensorflow as tf
import numpy as np
from datetime import datetime
from sklearn.datasets import fetch_california_housing


housing = fetch_california_housing()
m, n = housing.data.shape

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

data = housing.data
scaled_housing_data_plus_bias = (data-np.mean(data,axis=0))/np.std(data,axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data_plus_bias]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# 定义写入的东西
tf.summary.scalar('mse',mse)
tf.summary.histogram('theta',theta)

# 进行汇总
merge_summary=tf.summary.merge_all(key=tf.GraphKeys.SUMMARIES)

# 定义写入地址
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
# 打印写入的地址方便tensorboard使用
print(logdir)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer() 
with tf.Session() as sess:
    
    sess.run(init)
    # 定义写的文件（打开）
    file_writer=tf.summary.FileWriter(logdir,sess.graph)
    for epoch in range(n_epochs):
        if epoch%100==0: # 每隔一段时间
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
         # 计算写入的值
        summary_str = merge_summary.eval()
        file_writer.add_summary(summary_str, epoch) #epoch相当于横坐标
    best_theta = theta.eval()
    print(best_theta)
file_writer.close()

---

### First NN project demo

use mnist to do image clasiification.


In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import sys


tf.reset_default_graph()
epochs = 15
batch_size = 100
total_sum = 0
epoch = 0

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
train_num = mnist.train.num_examples



input_data = tf.placeholder(tf.float32,shape=(None,784))
input_label = tf.placeholder(tf.float32,shape=(None,10))

w1 = tf.get_variable(shape=(784,64),name='hidden_1_w')
b1 = tf.get_variable(shape=(64),initializer=tf.zeros_initializer(),name='hidden_1_b')

w2 = tf.get_variable(shape=(64,32),name='hidden_2_w')
b2 = tf.get_variable(shape=(32),initializer=tf.zeros_initializer(),name='hidden_2_b')

w3 = tf.get_variable(shape=(32,10),name='layer_output')

#logit层
output = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(input_data,w1)+b1),w2)+b2),w3)

loss = tf.losses.softmax_cross_entropy(input_label,output)

#opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
opt = tf.train.AdamOptimizer()

train_op = opt.minimize(loss)

# 测试评估
correct_pred = tf.equal(tf.argmax(input_label,axis=1),tf.argmax(output,axis=1))
acc = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

tf.add_to_collection('my_op',input_data)
tf.add_to_collection('my_op',output)
tf.add_to_collection('my_op',loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run([init])
    test_data = mnist.test.images
    test_label = mnist.test.labels
    while epoch<epochs:
        data,label=mnist.train.next_batch(batch_size)
        data = data.reshape(-1,784)
        total_sum+=batch_size
        sess.run([train_op],feed_dict={input_data:data,input_label:label})
        if total_sum//train_num>epoch:
            epoch = total_sum//train_num
            loss_val = sess.run([loss],feed_dict={input_data:data,input_label:label})
            acc_test = sess.run([acc],feed_dict={input_data:test_data,input_label:test_label})
            saver.save(sess, save_path="./model/my_model.ckpt")
            print('epoch:{},train_loss:{:.4f},test_acc:{:.4f}'.format(epoch,loss_val[0],acc_test[0]))